# PCA Tutorial

In this tutorial, we will learn about principal component analysis (PCA), and how it can be used to represent to generate low-dimensional representations of microstructures.

In [ ]:
import pymks
import numpy as np

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

First, we need to generate microstructures to work with. We'll generate 3 classes of structures: equiaxed, horizontal fibers, and vertical fibers.

In [ ]:
from pymks.datasets import make_microstructure
import numpy as np

X_1 = make_microstructure(n_samples=30, grain_size=(25, 25))
X_2 = make_microstructure(n_samples=30, grain_size=(15, 95))
X_3 = make_microstructure(n_samples=30, grain_size=(95, 15))

X = np.concatenate((X_1, X_2, X_3))

Now that we've generated our microstructures, we calculate their two point statistics.

In [ ]:
from pymks import PrimitiveBasis
from pymks.stats import correlate

p_basis = PrimitiveBasis(n_states=2, domain=[0, 1])
X_corr = correlate(X, p_basis, periodic_axes=[0, 1])
X_corr=X_corr[:,:,:,0].reshape(X.shape[0],X.shape[1]*X.shape[2])

PCA works via a factorization known as singular value decomposition (SVD). However, the mean of the input matrix must be subtracted first.

In [ ]:
X_meanshift=X_corr-X_corr.mean(axis=0)
u,s,v=np.linalg.svd(X_meanshift,full_matrices=False)
scores=u*s
basis=v

Basis vectors define the axes of our new coordinate system. They are ordered by decreasing captured variance of the input data. Therefore, low order basis vectors are more important for representing the data compared to high order basis vectors. Let's take a look at the first few basis vectors that we obtained in the previous step.

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    plt.figure()
    plt.imshow(basis[i].reshape(X.shape[1],X.shape[2]),
               extent=[-X.shape[1]/2,X.shape[1]/2,-X.shape[2]/2,X.shape[1]/2])
    plt.title('Basis vector {}'.format(i+1))
    plt.grid(False)

Let's observe the captured variance by basis vector:

In [ ]:
cumvar=np.var(scores,axis=0)
cumvar/=np.sum(cumvar)

plt.figure()
plt.bar(np.arange(1,91),cumvar)
plt.ylim(0,1)
plt.grid(False)
plt.xlabel('PC score')
plt.ylabel('Fraction of captured variance')

We can see that the first basis vector is dominant, with most other vectors providing negligible effect. This is largely because the first basis vector is highly related to volume fraction. Therefore, reducing the variation in a dataset's volume fractions is one way to reduce the dominance of the first basis vector. Now let's discuss PC scores. PC scores (or weights) represent the weight of a basis vector present in a structure. Let's plot the first two PC scores for our dataset and see what happens.

In [ ]:
pc_choice_1=1 #1st PC index to be plotted
pc_choice_2=2 #2nd PC index to be plotted

plt.figure()
plt.plot(scores[0:30,pc_choice_1-1],scores[0:30,pc_choice_2-1],'b.')
plt.plot(scores[30:60,pc_choice_1-1],scores[30:60,pc_choice_2-1],'r.')
plt.plot(scores[60:90,pc_choice_1-1],scores[60:90,pc_choice_2-1],'g.')
plt.grid(False)
plt.xlabel('PC{}'.format(pc_choice_1))
plt.ylabel('PC{}'.format(pc_choice_2))
plt.legend(['Equiaxed','Fibers (H)','Fibers (V)'])
plt.title('Representation of structures in PC space')

We obtain a very interesting result! The three classes of microstructures form three distinct clusters in the PC2 dimension, despite us not telling the code about the classes. Therefore, we may conclude that PC2 is capturing microstructure shape and orientation information. PCA is an excellent method for showing microstructure similarities in a low dimensional space. Now let's show that basis vectors and PC scores can be used to reconstruct the original two point statistics. 

In [ ]:
structure_selection=80
n_scores=20

tps_approx=np.matmul(scores[:,0:n_scores],basis[0:n_scores])+X_corr.mean(axis=0)

plt.subplot(1,2,1)
plt.imshow(X_corr[structure_selection].reshape(101,101),
           extent=[-X.shape[1]/2,X.shape[1]/2,-X.shape[2]/2,X.shape[1]/2])
plt.title('Full autocorrelations')

plt.grid(False)

plt.subplot(1,2,2)
plt.imshow(tps_approx[structure_selection].reshape(101,101),
           extent=[-X.shape[1]/2,X.shape[1]/2,-X.shape[2]/2,X.shape[1]/2])
plt.title('Approximated autocorrelations\n{} PC scores'.format(n_scores))

plt.grid(False)

plt.tight_layout()
plt.show()

app_error=np.round(100*np.sum(abs(X_corr[structure_selection]-tps_approx[structure_selection]))/np.sum(X_corr[structure_selection]),2)

print('The percent mean absolute error of the approximation is {}%.'.format(app_error))

The more PC scores/basis vectors are included, the more accurate the approximation of the two point statistics will be. But even with a few PC scores/basis vectors, we can obtain approximations within a few points of error. In this way, we are able to reduce very high dimensional correlations to a small number of PC scores with little loss of accuracy.